In [13]:
# here's where i will define and package my 
# layer space. 
import numpy as np

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input, add, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Convolution2D, MaxPooling2D

In [10]:
def conv_block(filter_depth, filter_size, pool_size, activation, inputs):
    x = BatchNormalization()(inputs)
    x = Convolution2D(filter_depth, filter_size, activation=activation)(x)
    x = MaxPooling2D(pool_size=pool_size)(x)
    return x

def dense_block(units, activation, drop_prob, inputs):
    x = BatchNormalization()(inputs)
    x = Dense(units, activation=activation)(x)
    x = Dropout(drop_prob)(x)
    return x

def resnet_block(filter_depth, filter_size, pool_size, activation, inputs):
    x = BatchNormalization()(inputs)
    x = Convolution2D(filter_depth, (1,1), activation=activation)(x)

    x = BatchNormalization()(x)
    x = Convolution2D(filter_depth, filter_size, activation=activation, padding='same')(x)
    
    x = BatchNormalization()(x)
    x = Convolution2D(filter_depth, (1,1))(x)
    
    shortcut = Convolution2D(filter_depth, (1,1))(inputs)
    shortcut = BatchNormalization()(shortcut)
    
    x = add([x, shortcut])
    x = Activation('relu')(x)
    return x

In [66]:
# semi randomly create a model, each input is of format:
# [model, (range), (range)] where ranges are for the hyperparameters
# -> range = (range_start, range_end, range_incrementer), range_incrementer
# is a method (labmda func) that tells you how to increment that hyper param
def rand_model(*args, blocks=1, in_shape):
    m = Input(shape=in_shape)
    inputs = m
    for i in range(blocks):
        params = []
        block = np.random.choice(args,1)
        args 
        for param_options in block[0][1:]:
            ix = np.random.choice(len(param_options),1)
            params.append(param_options[ix[0]])
        m = block[0][0](*params, inputs=m)
    
    output = dense_block(1, 'sigmoid', 0, m)
    
    model = Model(inputs=inputs, outputs=output)
    return model
    
conv_param_space = [conv_block, 
                    [4,256,512,1024,2048],
                    [(1,1),(2,2),(3,3),(4,4),(5,5)],
                    [(2,2),(3,3)],
                    ['relu']
                    ]
resnet_param_space = [resnet_block,
                    [16,32,64,128,256],
                    [(1,1),(2,2),(3,3),(4,4),(5,5)],
                    [(2,2),(3,3)],
                    ['relu']
                    ]
dense_param_space = [dense_block,
                     [256,512,1024,2048],
                     ['relu'],
                     [0.25,0.35,0.45,0.55,0.65,0.75]
                    ]

In [69]:
# make 5 2block models
models = []
for i in range(5):
    models.append(rand_model(conv_param_space, resnet_param_space, dense_param_space, blocks=2, in_shape=(128,128,3)))

In [ ]:
for 
script = '''#!/bin/bash

#SBATCH --job-name=jupyterGPU
#SBATCH --nodes=1
#SBATCH --cpus-per-task=2
#SBATCH --mem=64GB
#SBATCH --time=100:00:00
#SBATCH --gres=gpu:1
#SBATCH --job-name=optimizing_model
#SBATCH --mail-type=END
#SBATCH --mail-user=yns207@nyu.edu
#SBATCH --output=/scratch/yns207/job_logs/optimizing_model_%j.out
#SBATCH --error=/scratch/yns207/job_logs/optimizing_model_%j.err
#SBATCH --get-user-env

module purge
module load cuda
module load cudnn

unset XDG_RUNTIME_DIR
if [ "$SLURM_JOBTMP" != "" ]; then
    export XDG_RUNTIME_DIR=$SLURM_JOBTMP
fi

echo "optimizing model..."
time $HOME/anaconda3/envs/smappcollector/bin/python $HOME/nbs/machine_learning/gpu-rancher/fit_model.py -i {}
echo "done"
'''.format()